In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
client_info = es.info()
print ("Connectd to Elastic Search")
pprint(client_info.body)

Connectd to Elastic Search
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'WpOI-sfBSXe9aaVkWGIQnQ',
 'name': '61929d733ddf',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-12-16T10:09:08.849001802Z',
             'build_flavor': 'default',
             'build_hash': 'd8972a71dbbd64ff17f2f4dba9ca2c3fe09fb100',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.3.2',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.19.0',
             'number': '9.2.3'}}


In [3]:
es.indices.delete(index='my_index', ignore=[400, 404])
es.indices.create(index='my_index')

/tmp/ipykernel_1486/3702790224.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index='my_index', ignore=[400, 404])


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

## Insert one document

In [4]:
doc1 = {
    "title": "First Document",
    "content": "This is the content of the first document.",
    "timestamp": "2024-06-01T12:00:00"
}
response1 = es.index(index="my_index", document=doc1)


In [5]:
pprint(response1.body)

{'_id': '1ZfC7psBPICzm6Zx6LcE',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 0,
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_version': 1,
 'result': 'created'}


In [6]:
print("Document indexed, ID:", response1['_id'])
print("Index name:", response1['_index'])
print("Result:", response1['result'])
print("Version:", response1['_version'])
print("Shards info:", response1['_shards'])

Document indexed, ID: 1ZfC7psBPICzm6Zx6LcE
Index name: my_index
Result: created
Version: 1
Shards info: {'total': 2, 'successful': 1, 'failed': 0}


## Insert multiple documents

In [ ]:
import json

data = json.load(open('../data/dummy_data.json'))
data

[{'title': 'Sample Title 1',
  'text': 'This is the first sample document text.',
  'created_on': '2024-09-22'},
 {'title': 'Sample Title 2',
  'text': 'Here is another example of a document.',
  'created_on': '2024-09-24'},
 {'title': 'Sample Title 3',
  'text': 'The content of the third document goes here.',
  'created_on': '2024-09-24'}]

In [8]:
def insert_document(doc):
    response = es.index(index='my_index',   document=doc)
    return response

def print_response_info(response):
    print(f"""Document ID: {response['_id']} is '{
          response["result"]}' and is split into {response['_shards']['total']} shards.""")


In [9]:
for doc in data:
    response = insert_document(doc)
    print_response_info(response)

Document ID: 1pfC7psBPICzm6Zx-bet is 'created' and is split into 2 shards.
Document ID: 15fC7psBPICzm6Zx-rca is 'created' and is split into 2 shards.
Document ID: 2JfC7psBPICzm6Zx-rcn is 'created' and is split into 2 shards.


## print Mapping

In [14]:
index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping.body)

{'my_index': {'mappings': {'properties': {'content': {'fields': {'keyword': {'ignore_above': 256,
                                                                             'type': 'keyword'}},
                                                      'type': 'text'},
                                          'created_on': {'type': 'date'},
                                          'text': {'fields': {'keyword': {'ignore_above': 256,
                                                                          'type': 'keyword'}},
                                                   'type': 'text'},
                                          'timestamp': {'type': 'date'},
                                          'title': {'fields': {'keyword': {'ignore_above': 256,
                                                                           'type': 'keyword'}},
                                                    'type': 'text'}}}}}


In [15]:
pprint(index_mapping['my_index']['mappings'])

{'properties': {'content': {'fields': {'keyword': {'ignore_above': 256,
                                                   'type': 'keyword'}},
                            'type': 'text'},
                'created_on': {'type': 'date'},
                'text': {'fields': {'keyword': {'ignore_above': 256,
                                                'type': 'keyword'}},
                         'type': 'text'},
                'timestamp': {'type': 'date'},
                'title': {'fields': {'keyword': {'ignore_above': 256,
                                                 'type': 'keyword'}},
                          'type': 'text'}}}


## Manual Mapping

In [16]:
es.indices.delete(index='my_index', ignore_unavailable=True )
es.indices.create(index='my_index')

mapping = {
    "properties": {
        "title": {
            "type": "text",
            "fields": {
                "keyword": {
                    "type": "keyword",
                    "ignore_above": 256
                    }
                }
            },
        "text": {"type": "text"},
        "created_on": {"type": "date"}
    }
}

es.indices.put_mapping(index='my_index', body=mapping)

ObjectApiResponse({'acknowledged': True})

In [19]:
# another method
es.indices.delete(index='my_index', ignore_unavailable=True )
es.indices.create(index='my_index', mappings=mapping )

index_mapping = es.indices.get_mapping(index='my_index')
pprint(index_mapping['my_index']['mappings']['properties'])

{'created_on': {'type': 'date'},
 'text': {'type': 'text'},
 'title': {'fields': {'keyword': {'ignore_above': 256, 'type': 'keyword'}},
           'type': 'text'}}
